In [53]:
import csv
import math
import random




In [54]:
with open('Colonia.csv', newline='',encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    cities = []
    for row in reader:
        city = row['Cidade']
        x, y = map(int, (row['X'], row['Y']))
        cities.append((city, x, y))


In [55]:
def distCalculator(ponto1, ponto2):
    return math.sqrt((ponto1[1]-ponto2[1])**2 + (ponto1[2]-ponto2[2])**2)


In [56]:
grafo = []
initialPhero=1
for i in cities:
    arestas = []
    for j in cities:
        dist = distCalculator(i,j)
        if dist == 0:
            inverseDist = 0
        else:
            inverseDist = 1/distCalculator(i,j)
        arestas.append({
            'dist': dist,
            'phero': initialPhero,
            'invDist': inverseDist,
            'heuristic': initialPhero * inverseDist,
            'prob': 0
        })
    grafo.append(arestas)

grafo


[[{'dist': 0.0, 'phero': 1, 'invDist': 0, 'heuristic': 0, 'prob': 0},
  {'dist': 4.0, 'phero': 1, 'invDist': 0.25, 'heuristic': 0.25, 'prob': 0},
  {'dist': 8.0, 'phero': 1, 'invDist': 0.125, 'heuristic': 0.125, 'prob': 0},
  {'dist': 4.123105625617661,
   'phero': 1,
   'invDist': 0.24253562503633297,
   'heuristic': 0.24253562503633297,
   'prob': 0},
  {'dist': 2.23606797749979,
   'phero': 1,
   'invDist': 0.4472135954999579,
   'heuristic': 0.4472135954999579,
   'prob': 0},
  {'dist': 2.23606797749979,
   'phero': 1,
   'invDist': 0.4472135954999579,
   'heuristic': 0.4472135954999579,
   'prob': 0},
  {'dist': 5.0990195135927845,
   'phero': 1,
   'invDist': 0.19611613513818404,
   'heuristic': 0.19611613513818404,
   'prob': 0},
  {'dist': 2.0, 'phero': 1, 'invDist': 0.5, 'heuristic': 0.5, 'prob': 0},
  {'dist': 2.8284271247461903,
   'phero': 1,
   'invDist': 0.35355339059327373,
   'heuristic': 0.35355339059327373,
   'prob': 0},
  {'dist': 4.47213595499958,
   'phero': 1,
  

In [57]:
def atualizaHeur(grafo):
    alpha = 3.5
    beta = 4.4
    for i in grafo:
        for j in i:
            j['heuristic'] = (j['phero']**alpha) * (j['invDist']**beta)

def atualizaProb(grafo):
    for i in grafo:
        somaHeuristica = 0
        for j in i:
            somaHeuristica += j['heuristic']
        for j in i:
            j['prob']= j['heuristic']/somaHeuristica


In [58]:
atualizaProb(grafo)



In [59]:
def roleta(cidadeAtual,javisitados):
    roleta = []
    soma = 0
    for j in range((len(cities))):
        if j not in javisitados:
            roleta.append((j,grafo[cidadeAtual][j]['prob']+soma))
            soma+=grafo[cidadeAtual][j]['prob']
    gerado = random.uniform(0,roleta[-1][1])
    for j in roleta:
        if gerado <= j[1]:
            return j[0]


In [60]:
def caminhar():
    pathsAndDist = []
    for i in range(len(cities)):
        javisitados = {i:1}
        cidadeAtual = i
        distanciaPercorrida = 0
        path = [i]
        while (len(javisitados)<len(cities)):
            selecionado = roleta(cidadeAtual,javisitados)
            javisitados[selecionado] = 1
            path.append(selecionado)
            distanciaPercorrida += grafo[cidadeAtual][selecionado]['dist']
            cidadeAtual = selecionado

        distanciaPercorrida+= grafo[cidadeAtual][i]['dist']
        path.append(i)
        pathsAndDist.append((path,distanciaPercorrida))
    return pathsAndDist

In [61]:
def atualizaFero(pathsAndDist):
    evaporacao = 0.005
    pheroAmount = 10
    for i in pathsAndDist:
        path = i[0]
        dist = i[1]
        pheroPorRota = pheroAmount/dist
        for j in range(1,len(path)):
            grafo[path[j]][path[j-1]]['phero'] = (1-evaporacao)*grafo[path[j]][path[j-1]]['phero'] + pheroPorRota
            grafo[path[j-1]][path[j]]['phero'] = (1-evaporacao)*grafo[path[j-1]][path[j]]['phero'] + pheroPorRota
            



In [65]:
data = []
for i in range(300):
    pathsAndDist = caminhar()
    print(pathsAndDist)
    data.append(sum([i[1] for i in pathsAndDist])/len(pathsAndDist))
    atualizaFero(pathsAndDist)
    atualizaHeur(grafo)
    atualizaProb(grafo)

data
    
    

[([0, 4, 7, 13, 12, 16, 23, 27, 24, 21, 31, 28, 25, 22, 29, 26, 30, 20, 19, 11, 3, 8, 5, 14, 9, 15, 17, 10, 18, 1, 6, 2, 0], 83.11461638169372), ([1, 6, 9, 8, 5, 7, 13, 16, 23, 27, 24, 21, 28, 31, 25, 29, 22, 18, 15, 17, 14, 0, 3, 4, 20, 19, 26, 30, 12, 11, 2, 10, 1], 78.50151384084397), ([2, 10, 9, 1, 6, 15, 17, 18, 22, 29, 25, 28, 31, 23, 27, 24, 21, 14, 8, 5, 7, 13, 16, 12, 0, 11, 19, 26, 20, 30, 4, 3, 2], 81.20935313160491), ([3, 13, 7, 12, 16, 24, 27, 23, 20, 26, 19, 11, 4, 0, 5, 8, 9, 15, 17, 25, 29, 31, 28, 21, 14, 6, 1, 10, 2, 22, 18, 30, 3], 80.62286171506427), ([4, 3, 11, 19, 20, 16, 12, 13, 8, 5, 0, 7, 14, 9, 6, 1, 15, 21, 24, 27, 23, 30, 26, 28, 29, 31, 25, 22, 18, 10, 2, 17, 4], 77.09342536480237), ([5, 8, 0, 7, 12, 4, 19, 26, 20, 11, 3, 13, 16, 23, 27, 24, 21, 28, 31, 29, 25, 18, 22, 17, 15, 6, 9, 14, 1, 10, 2, 30, 5], 82.87885663863605), ([6, 9, 15, 17, 14, 5, 1, 2, 10, 18, 22, 25, 29, 28, 31, 21, 24, 27, 23, 30, 26, 20, 19, 11, 16, 13, 12, 7, 3, 4, 0, 8, 6], 67.38445474

[78.83471563542714,
 80.68274691190841,
 79.14001517869593,
 78.75568060689419,
 79.24489372136473,
 81.57494711337895,
 80.7857949850168,
 80.3204845819812,
 79.87875416417461,
 79.76272058898077,
 79.57973206084107,
 80.03958529271897,
 80.43571439217332,
 81.835638750978,
 79.98863956045805,
 81.9659218038751,
 81.0622424780464,
 80.3213856471677,
 81.43015456371907,
 80.74901391384674,
 80.57817841145946,
 80.98344080254262,
 80.83735516939495,
 78.53958743383257,
 79.08242187677816,
 80.48330142076927,
 82.0684652573572,
 80.5299546472904,
 79.1192078599729,
 81.60902879419304,
 81.38312204040433,
 79.56998988125675,
 80.61288999780919,
 81.52157800379103,
 80.37622230319575,
 81.29831502948782,
 78.98805428014701,
 81.04332724680057,
 79.44804478124888,
 81.3707036402551,
 79.66220754943693,
 81.17392277189403,
 80.09783787401483,
 79.78600928822233,
 81.17235241041186,
 82.95280426309677,
 81.92014376934566,
 81.61252198522453,
 80.30470988663247,
 80.11474265972947,
 80.0476597